### Imports

In [1]:
# base and metaprogramming imports
from __future__ import annotations
import os
import shutil
import sys
import itertools as it
import pprint as pp
from typing import Optional
import warnings

# analysis imports
import h5py
import numpy as np
import pandas as pd
import pandas
from pandas.api.types import CategoricalDtype
import scipy
import scipy.ndimage
import skimage as ski
import ants

# visualization imports
import napari
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns

# ipynb-specific imports
try:
    import ipynbname
except Exception as e:
    warnings.warn(f'`ipynbname` module import failed with error: {e!r}')
    ipynbname = None
    
if ipynbname is not None:
    _file = ipynbname.path()
else:
    # manually set the full path to the current notebook below if `ipynbname` 
    # import fails
    _file = ''
    
jobs_dir = os.path.split(_file)[0]
sources_dir = os.path.abspath(os.path.join(jobs_dir, '..'))
repo_dir = os.path.abspath(os.path.join(sources_dir, '..'))
config_dir = os.path.abspath(os.path.join(repo_dir, 'configs'))

# add sources directory to path
if sources_dir not in sys.path:
    sys.path.append(sources_dir)

# local imports
import c_swain_python_utils as csutils
from utilities import *
import imaging_dataset as imd
import prairie_view_imports as pvi

_basename = os.path.basename(_file)
_name = csutils.no_ext_basename(_file)


# Table Conversion
## Import Table

In [8]:
data_dir = os.path.join(repo_dir, 'data')
tsv_filename = 'olfac-stim-compare-table.tsv'
tsv_path = os.path.join(data_dir, tsv_filename)

df = pandas.read_csv(tsv_path, sep='\t', header=0)
df

,Reference (DOI),Year,Number of Odor Channels,Number of Stimulation Directions,Fish Age,Fish State,Realtime/Continuous Flow,Simultaneous Behavioral Imaging,Simultaneous Brain Ca Imaging,Bilateral Discrimination,Device Platform,Additional Stimulation Modalities
0,\citeauthor{maaswinkel:2003:j.exp.biol.},\citeyear{maaswinkel:2003:j.exp.biol.},1,1,4--12 month,free swimming,YES,YES,NO,NO,circular container with rotating drum visual s...,visual
1,\citeauthor{bhinder:2012:springerprot}\cite{bh...,\citeyear{bhinder:2012:springerprot},1,1,>\~2 week,free swimming,YES,YES,NO,NO,4 quadrant ring shaped tank,food application
2,\citeauthor{braubach:2009:behav.brain.res.}\ci...,\citeyear{braubach:2009:behav.brain.res.},1,0,3--7 day,free swimming,NO,YES,NO,NO,96-well plate wells,NaN
3,\citeauthor{biechl:2016:sci:rep}\cite{biechl:2...,\citeyear{biechl:2016:sci:rep},2,2,9 day,free swimming,YES,YES,NO,NO,2 chanel choice flume,NaN
4,\citeauthor{jeong:2021:int.j.mol.sci.}\cite{je...,\citeyear{jeong:2021:int.j.mol.sci.},1,1,5 day,free swimming,YES,YES,NO,NO,cross shaped maze,optogenetic activation
5,\citeauthor{sy:19}\cite{sy:19},\citeyear{sy:19},2,2,5--7 day,"head-fixed, tail-free",YES,YES,YES,YES,microfluidic chip,NaN
6,\citeauthor{herrera:21}\cite{herrera:21},\citeyear{herrera:21},8,1,5--7 day,"head-fixed, tail-free",YES,YES,YES,YES,agarose-embedding in dish,NaN
7,this work,2024,10,3,6--7 day,"head-fixed, tail-free",YES,YES,YES,YES,agarose-embedding in dish,NaN


In [9]:
styler = (
    df
    .style
    .format({
        'Number of Odor Channels': '{:2d}',
        'Number of Stimulation Directions': '{:2d}'})
    .map(lambda )
)

styler

,Reference (DOI),Year,Number of Odor Channels,Number of Stimulation Directions,Fish Age,Fish State,Realtime/Continuous Flow,Simultaneous Behavioral Imaging,Simultaneous Brain Ca Imaging,Bilateral Discrimination,Device Platform,Additional Stimulation Modalities
0,\citeauthor{maaswinkel:2003:j.exp.biol.},\citeyear{maaswinkel:2003:j.exp.biol.},1,1,4--12 month,free swimming,YES,YES,NO,NO,circular container with rotating drum visual stimuli,visual
1,\citeauthor{bhinder:2012:springerprot}\cite{bhinder:2012:springerprot},\citeyear{bhinder:2012:springerprot},1,1,>\~2 week,free swimming,YES,YES,NO,NO,4 quadrant ring shaped tank,food application
2,\citeauthor{braubach:2009:behav.brain.res.}\cite{braubach:2009:behav.brain.res.},\citeyear{braubach:2009:behav.brain.res.},1,0,3--7 day,free swimming,NO,YES,NO,NO,96-well plate wells,nan
3,\citeauthor{biechl:2016:sci:rep}\cite{biechl:2016:sci:rep},\citeyear{biechl:2016:sci:rep},2,2,9 day,free swimming,YES,YES,NO,NO,2 chanel choice flume,nan
4,\citeauthor{jeong:2021:int.j.mol.sci.}\cite{jeong:2021:int.j.mol.sci.},\citeyear{jeong:2021:int.j.mol.sci.},1,1,5 day,free swimming,YES,YES,NO,NO,cross shaped maze,optogenetic activation
5,\citeauthor{sy:19}\cite{sy:19},\citeyear{sy:19},2,2,5--7 day,"head-fixed, tail-free",YES,YES,YES,YES,microfluidic chip,nan
6,\citeauthor{herrera:21}\cite{herrera:21},\citeyear{herrera:21},8,1,5--7 day,"head-fixed, tail-free",YES,YES,YES,YES,agarose-embedding in dish,nan
7,this work,2024,10,3,6--7 day,"head-fixed, tail-free",YES,YES,YES,YES,agarose-embedding in dish,nan


In [10]:
output_dir = os.path.join(data_dir, 'outputs')
csutils.touchdir(output_dir)
original_name, _ = os.path.splitext(tsv_filename)
output_path = os.path.join(output_dir, f'FORMATTED_{original_name}.tex')

styler.to_latex(
    buf=output_path,
    siunitx=True,
    hrules=True,
    caption='Comparison many different olfactory stimulation technologies to and their features to this present work.')